# Define the problem

**Process Management Problem**

In the process management problem, we are given a set of tasks represented by processes $p$. Each process $p_i$ has an associated value $v_i$​ (indicating its importance) and a duration $d_i$​ (representing the time required to execute the task). Additionally, there exists a maximum allowed duration $d_{max}$​ within which we must execute these tasks.


Our objective is to select a subset $S$ of processes from $p$ such that their total duration does not exceed $d_{max}$​, while maximizing the overall value of the executed processes. In other words, we aim to find the set S that maximizes the expression:

$$\sum_{i\in S} v_i$$

subject to the constraint:

$$\sum_{i\in S} d_i \leq d_{max}$$

## Define problem instance

In [1]:
import random 


random.seed(13)

# Define the number of items
n_items = 6

# Define ranges
duration_range = [1, 7]
values_range = [5, 15]
max_duration_percentage = 0.7

# Fill the weights and values 
duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]

# Compute the maximum allowed weight
max_duration = int(max_duration_percentage * sum(duration))


# Print the instance
print("-" * 20)
print("Instance Details:")
print("-" * 20)
print(f"Duration                 : {duration}")
print(f"Values                   : {values}")
print(f"Total duration           : {sum(duration)}")
print(f"Maximum allowed duration : {max_duration}")

--------------------
Instance Details:
--------------------
Duration                 : [3, 3, 6, 6, 7, 7]
Values                   : [7, 15, 8, 15, 7, 8]
Total duration           : 32
Maximum allowed duration : 22


Reference Solution (Selected Processes are **Bolded**):

Duration: [3, **3**, **6**, **6**, 7, **7**]

Values: [7, **15**, **8**, **15**, 7, **8**]

Total selected Duration: 22

Total selected values: 46

# Hackathon Tasks 


## Problem 1

### Build the cost hamiltonians for the Process Management problem:

$y_i$ are slack variables representing the total duration for the selected solution.

$x_i$ encodes the solution as a bit string.

$d_i$ are the durations, $v_i$ are the values, $D$ is the maximum duration.

$$
C = A \Big( 1 - \sum_{i=1}^D y_i \Big)^2 + A \Big( \sum_{i=1}^D i y_i - \sum_{i=1}^n x_i d_i \Big)^2 - B \sum_{i=1}^n x_i v_i
$$

We use the $O = \frac{I + Z}{2}$ operator.

$$
H = A \Big( 1 - \sum_{i=1}^D \frac{1 - Z(y_i)}{2} \Big)^2 + A \Big( \sum_{i=1}^D i \frac{1 - Z(y_i)}{2} - \sum_{i=1}^n \frac{1 - Z(x_i)}{2} d_i \Big)^2 - B \sum_{i=1}^n \frac{1 - Z(x_i)}{2} v_i
$$

We set $B = 1$ and $A = \max_i(v_i)$.

In [111]:
from qibo.symbols import Z, X
from qibo.hamiltonians import SymbolicHamiltonian

def build_cost_hamiltonian_too_many_qubits(values: list[int], duration: list[int], max_duration: int) -> SymbolicHamiltonian:
    """This function should be filled to build the problem cost hamiltonian.

    Args:
        values (list[int]): the list of values.
        duration (list[int]): the list of durations. 
        max_duration (int): the maximum value of the allowed duration. 
    """

    # By convention:
    #   - the qubits 0, ..., D - 1 are the slack variables (y_i).
    #   - the qubits D, ..., D + n - 1 are the solution variables (x_i).
    slack_to_qubit = {
        i: i - 1
        for i in range(1, max_duration + 1)
    }
    solution_to_qubit = {
        i: max_duration + i - 1
        for i in range(1, len(values) + 1)
    }
    B = 1
    A = max(values)

    term_1 = 1 - sum(
        (1 - Z(slack_to_qubit[i]) / 2)
        for i in range(1, max_duration + 1)
    )
    term_2 = sum(
        i * (1 - Z(slack_to_qubit[i]) / 2)
        for i in range(1, max_duration + 1)
    ) - sum(
        duration[i - 1] * (1 - Z(solution_to_qubit[i]) / 2)
        for i in range(1, len(values) + 1)
    )
    term_3 = sum(
        values[i - 1] * (1 - Z(solution_to_qubit[i])) / 2
        for i in range(1, len(values) + 1)
    )
    cost_hamiltonian = A * term_1 ** 2 + A * term_2 ** 2 - B * term_3
    

    return SymbolicHamiltonian(cost_hamiltonian)

But this requires a lot of qubits (28), which is untractable.

Instead, we use the “log trick”, to encode the “total duration” with binary encoding, instead of “one-shot” encoding. We define $M = \lfloor \ln(D) \rfloor + 1$.

$$
C = A \Big( \sum_{i=0}^{M-1} 2^i y_{i+1} + (D + 1 - 2^M) y_{M} - \sum_{i=1}^n x_i d_i \Big)^2 - B \sum_{i=1}^n x_i v_i
$$

$$
H = A \Big( \sum_{i = 0}^{M - 1} 2^i \frac{1 - Z(y_{i+1})}{2} + (D + 1 - 2^M) \frac{1 - Z(y_M)}{2} - \sum_{i=1}^n \frac{1 - Z(x_i)}{2} d_i \Big)^2 - B \sum_{i=1}^n \frac{1 - Z(x_i)}{2} v_i
$$

In [106]:
import math

def build_cost_hamiltonian(values: list[int], duration: list[int], max_duration: int) -> SymbolicHamiltonian:
    """This function should be filled to build the problem cost hamiltonian.

    Args:
        values (list[int]): the list of values.
        duration (list[int]): the list of durations. 
        max_duration (int): the maximum value of the allowed duration. 
    """

    # By convention:
    #   - the qubits 0, ..., M are the slack variables (y_i).
    #   - the qubits M + 1, ..., M + D are the solution variables (x_i).
    M = math.floor(math.log(max_duration))

    B = 1
    A = max(values)

    cost_hamiltonian = A * (
        sum(
            2**j * (1 - Z(j)) / 2
            for j in range(0, M)  # 0, ..., M - 1
        ) +
        (max_duration + 1 - 2**M) * (1 - Z(M)) / 2 -
        sum(
            duration[i] * (1 - Z(M + i + 1)) / 2
            for i in range(0, len(values))
        )
    )**2 - B * (
        sum(
            values[i] * (1 - Z(M + i + 1)) / 2
            for i in range(0, len(values))
        )
    )
    

    return SymbolicHamiltonian(cost_hamiltonian)

### Compute the number fo qubits required to execute the cost hamiltonian

In [109]:
# define the number of qubits:

nqubits= len(values) + math.floor(math.log(max_duration)) + 1

### Diagonalize the cost hamiltonian and examen its eigenvalues and eigenvectors. 
- Show that the ground state eigenvector does actually correspond to the ground state energy of the problem. (Note: the ground state could be degenerate)

In [110]:
import numpy as np
from qibo import set_backend

# set the backend used for the calculation 
set_backend("numpy", platform=None)


# create the cost Hamiltonian for the given graph
cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)

ham_matrix = cost_hamiltonian.matrix

eig_val, eig_vec = np.linalg.eig(ham_matrix)
eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]

vec = zip(eig_val, eig_vec)
diagonalized_solution = sorted(vec, key=lambda x: x[0])

print()
print(diagonalized_solution)

[Qibo 0.2.7|INFO|2024-05-04 15:20:57]: Using numpy backend on /CPU:0
[Qibo 0.2.7|WARNING|2024-05-04 15:20:57]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.



[((-46+0j), '1111011101'), ((-45+0j), '0011110101'), ((-45+0j), '1101111100'), ((-45+0j), '1111011110'), ((-44+0j), '0011110110'), ((-38+0j), '0001011100'), ((-38+0j), '0011111001'), ((-38+0j), '1001010101'), ((-38+0j), '1111101101'), ((-37+0j), '0011111010'), ((-37+0j), '1001010110'), ((-37+0j), '1011110011'), ((-37+0j), '1111101110'), ((-31+0j), '0011001101'), ((-31+0j), '0111011101'), ((-31+0j), '1001011001'), ((-30+0j), '0001101100'), ((-30+0j), '0011001110'), ((-30+0j), '0011111100'), ((-30+0j), '0101010011'), ((-30+0j), '0101111100'), ((-30+0j), '0111011110'), ((-30+0j), '1001011010'), ((-30+0j), '1001100101'), ((-30+0j), '1011000111'), ((-30+0j), '1011110101'), ((-30+0j), '1101110101'), ((-30+0j), '1111010111'), ((-29+0j), '1001100110'), ((-29+0j), '1011110110'), ((-29+0j), '1101110110'), ((-23+0j), '0001010101'), ((-23+0j), '0101010101'), ((-23+0j), '0111101101'), ((-23+0j), '1001011100'), ((-23+0j), '1001101001'), ((-23+0j), '1011001011'), ((-23+0j), '1011111001'), ((-23+0j),

## Problem 2

### Given that we only have **6 qubits** to solve this problem. Reduce the hamiltonian to use **only 6 qubits**.

In [36]:
nqubits = 6

### Solve this hamiltonian using the QAOA algorithm. 

        step 1. Define the Mixing hamiltonian.

In [ ]:
def build_mixer_hamiltonian(nqubits: int) -> SymbolicHamiltonian:
    '''
    build the mixer hamiltonian for the given graph.

    args:
        graph: graph
            A network graph

    returns:
        The mixer hamiltonian of the given graph

    '''
    mixing_hamiltonian = ... # TODO: define the Mixing Hamiltonian 
    
    return SymbolicHamiltonian(mixing_hamiltonian)

        Step 2. Run the QAOA algorithm.

In [ ]:
from qibo import models

hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
mixer_hamiltonian = build_mixer_hamiltonian(nqubits)

# create QAOA model given the Hamiltonians
qaoa = models.QAOA(hamiltonian=hamiltonian, mixer=mixer_hamiltonian)

# optimize using random initial variational parameters with four layers 
n_layers = 5
# initial_parameters = 0.01 * np.random.random(n_layers * 2)
initial_parameters =  0.01 *  (2 * np.random.random(n_layers * 2) - 1) * np.pi
print(initial_parameters)

# run the QAOA optimization with the initial parameters and the hamiltonians defined 

# Define random initial variational parameters with four layers 
n_layers = 4
initial_parameters = 0.01 * np.random.random(n_layers * 2)


"""
supported optimization Method: 
- Nelder-Mead 
- parallel_L-BFGS-B
- Powell
- CG
- cma
- sgd
- L-BFGS-B
- Newton-CG
- COBYLA
- BFGS
- trust-constr
"""
method = "Powell"


best_energy, final_parameters, _ = qaoa.minimize(initial_parameters, method=method)

print("best energy: ", best_energy)

    Step 3. Display solution and compare it to the ground state you obtained in problem 1

In [ ]:
qaoa.set_parameters(final_parameters)
quantum_state = qaoa.execute(None)

probabilities = (np.abs(quantum_state) ** 2)
probabilities = zip(probabilities, [i for i in range(len(probabilities))])
probabilities = sorted(probabilities, key=lambda x: x[0], reverse=True)
probabilities = [("{:.2f} %".format(round(p*100, 4)), "{0:0{bits}b}".format(s, bits=nqubits)) for p, s in probabilities]
print(probabilities)
qaoa_solution = probabilities[0][1]
print("-"*10)

print("The best solution is:", qaoa_solution )

### Solve this hamiltonian using Quantum Adiabatic Evolution

        Setp 1. Define initial hamiltonian

In [ ]:
def build_initial_hamiltonian(nqubits: int) -> SymbolicHamiltonian:
    '''
    build the mixer hamiltonian for the given graph.

    args:
        graph: graph
            A network graph

    returns:
        The mixer hamiltonian of the given graph

    '''
    mixing_hamiltonian = ... # TODO: define the Mixing Hamiltonian 
    
    return SymbolicHamiltonian(mixing_hamiltonian)

    Step 2. Run the Adiabatic Evolution

In [ ]:
import numpy as np
from qibo.models.evolution import AdiabaticEvolution
from qibo.hamiltonians.hamiltonians import Hamiltonian


# build initial (H0) and target (H1) hamiltonians
H0 = build_initial_hamiltonian(nqubits=nqubits)
H1 = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)

# calculate the dense hamiltonian from the symbolic hamiltonian
H1_dense = Hamiltonian(nqubits, H1.calculate_dense().matrix)
H0_dense = Hamiltonian(nqubits, H0.calculate_dense().matrix)

# Define the time steps
dt = 0.1

# Define the final evolution time
T = 20

# define the schedule. This is a function of time that defines the scheduling of the adiabatic evolution. 
# Can be either a function of time s(t) or a function with two arguments s(t, p) 
# where p corresponds to a vector of parameters to be optimized.
def s(t): return t

# construct the adiabatic model
adiabatic_model = AdiabaticEvolution(H0_dense, H1_dense, s, dt)

# execute the adiabatic optimization
adiabatic_quantum_state = adiabatic_model.execute(final_time=T)

    Step 3. Display solution and compare it to the ground state you obtained in problem 1 and using QAOA

In [ ]:
# obtain the state with the highest probability
solution_dec  = (np.abs(adiabatic_quantum_state) ** 2).argmax()
adiabatic_solution = "{0:0{bits}b}".format(solution_dec, bits=nqubits)


print("the solution is: ", adiabatic_solution)

### What are the downsides to the method you used to reduce the number of qubits? 

write your answer here...

## Problem 3

### Given this new dataset generate a hamiltonian that only uses 3 qubits 

In [4]:
import random 


random.seed(13)

# Define the number of items
n_items = 3

# Define ranges
max_duration_percentage = 0.7

# Fill the weights and values 
values   = [6, 2, 6]
duration = [3, 3, 6]

# Compute the maximum allowed weight
max_duration = int(max_duration_percentage * sum(duration))



# Print the instance
print("-" * 20)
print("Instance Details:")
print("-" * 20)
print(f"Duration                 : {duration}")
print(f"Values                   : {values}")
print(f"Total duration           : {sum(duration)}")
print(f"Maximum allowed duration : {max_duration}")

--------------------
Instance Details:
--------------------
Duration                 : [3, 3, 6]
Values                   : [6, 2, 6]
Total duration           : 12
Maximum allowed duration : 8


### Design a circuit layout that is suitable for this Hamiltonian

    Step 1. Create each element of the circuit

Your qubits will have a critical current of 230nA, shunting capacitance of 50fF, Z loop capacitance of 4.4fF, inductance of 480pH, x to z loop current ration of 0.4 and junction assymetry of 0.

The coupler's parameters are given

In [ ]:
import cas as cas

# coupler parameters
i_sigma = 565
c_sigma = 11
lc = 580
d = 0.0

# define couplers and qubits

circuit_elements = [...]

    Step 2. Define the qubit-coupler interaction and create the circuit object

A good approximation is to consider only the first 5 energy levels of the qubit and the first 2 energy levels of the coupler

In [ ]:
m = 65; m_mat = np.zeros((6, 6)) # mutual inductance matrix (given)

m_mat[0, 1] = -m; m_mat[1, 2] = m
m_mat[2, 3] = -m; m_mat[3, 4] = m
m_mat[4, 5] = -m; m_mat[5, 0] = m

m_mat = -(m_mat + m_mat.T)

trunc_vec = np.array([...]) # truncation vector for the energies of each element
circuit = cas.AnnealingCircuit(circuit_elements, m_mat, trunc_vec)

### Design an annealing Ising schedule 

    Step 1. Define the starting (initial) and objective hamiltonian coefficients

In [ ]:
# initialization hamiltonian
h0 = ... # type: dict[str, np.ndarray]

# objective hamiltonian
hf = ... # type: dict[str, np.ndarray]

    Step 2. Create a method to get a an arbitrary schedule for the circuit designed above

In [ ]:
def get_schedule(h0: dict[str, np.ndarray], hf: dict[str, np.ndarray], points: int, schedule: callable) -> dict[str, int | np.ndarray]:
    """Given the ising coefficients for the initial and final hamiltonian of the annealing scheduel,
    and given some time dependance of the schedule (eg. linear, exponential, etc), returns the ising
    coefficients for x and z at each time step.

    args:
        h0 (dict[str, np.ndarray]): ising coefficients of the starting hamiltonian h0
        hf (dict[str, np.ndarray]): ising coefficients of the target (final) hamiltonian hf
        points (int): number of points in the annealing schedule
        schedule (callable): time dependance of the schedule
    
    returns:
        dict[str, np.ndarray]: Dictionary of ising coefficients at each step of the schedule
    """

    ising_dict = ...
    
    return ...



    Step 2. Get the ising coefficients for a linear and an exponential schedule for a resolution of 1ns

In [ ]:
# linear schedule
linear_schedule = ... # feel free to use any callable (e.g. lambda function or method)
ising_linear = get_schedule(h0, hf, points=10, schedule=linear_schedule)
# exponential schedule
...

    Step 3. Calculate the flux schedule for the exponential schedule

In [ ]:
def get_fluxes(ising_schedule: dict[str, int | np.ndarray], optimizer: str) ->  dict[str, int | np.ndarray]:
    """Get the flux schedule from a given ising schedule.

    Args:
        ising_schedule (dict[str, int  |  np.ndarray]): dictionary with the number of points in the schedule
            and the ising coefficients for each control of the circuit at each point of the schedule
        optimizer (str): name of the lmfit minimizer to use

    Returns:
        dict[str, int | np.ndarray]: dictionary containing the number of points in the schedule and the fluxes
            to be applied at each control of the circuit at each point of the schedule.
    """
    flux_dict = ...
    return flux_dict
    

### With the obtain fluxes calculate the energy spectrum of the circuit

    Step 1. Define a function to get the spectrum from the flux schedule using the code from CAS

In [ ]:
def get_schedule_spectrum(flux_dict: dict[str, int | np.ndarray], levels: int) -> np.ndarray:
    """Calculate the energy spectrum through the schedule. The more levels we try to calculate,
    the more computationally expensive this will be.

    Args:
        flux_dict (dict[str, int  |  np.ndarray]): dictionary containing the number of points in the schedule and the fluxes
            to be applied at each control of the circuit at each point of the schedule.
        levels (int): energy levels to compute

    Returns:
        np.ndarray: of size (schedule points, levels) with the energy gaps for each level (En - E0),
            where E0 is the ground state energy and En is the energy of level n.
    """
    

### Research questions

1. For the annealing schedules calculated, do any of the resulting spectral schedules fulfill the adiabatic theorem?
2. By this point you will have obtained two energies, one from the software challenge (QAOA) and one from the annealing schedule above. Argue their relation.
3.  If you have made it here, you have problaly seen the difficulties of solving small instances. Can you propose ways to scale these procedures to more qubits? Some techniques for finding the gap along the annealing process can be found in [1], [2], [3] and [4]. Can you implement a simulation from one of these papers? Research and propose better techniques to find the flux schedule for a given Ising schedule with a greater amount of qubits.  

## Bibliography

[1] Adiabatic Spectroscopy and a Variational Quantum Adiabatic Algorithm: https://arxiv.org/abs/2103.01226

[2] Direct estimation of the energy gap between the ground state and excited state with quantum annealing: https://arxiv.org/abs/2007.10561

[3] Simulating quantum circuits by adiabatic computation: improved spectral gap bounds: https://arxiv.org/abs/1906.05233

[4] Spectroscopy on two coupled flux qubits: https://arxiv.org/abs/cond-mat/0308192

